In [168]:
import glob
import pandas as pd
import re
import json
import nltk
import numpy as np
import random

In [212]:
# Tagged lists of Activity/ object images from kayburns' Github Repo "Women Snowboard"
# https://github.com/kayburns/women-snowboard

activity_list_paths = glob.glob("./data/list/intersection_*")
activity_image_ids = dict()
for path in activity_list_paths:
    with open(path, 'r') as f:
        p = re.compile(r'(?<=_)(.*)(?=_)')
        activity = p.findall(path)[0]
        im_ids= f.read().split('\n')
        im_ids = [int(i) for i in im_ids if i != '']
        activity_image_ids[activity] = im_ids

activity_image_ids.keys()

dict_keys(['food', 'baseball', 'umbrella', 'kitchen', 'tie', 'snowboard', 'skateboard', 'table', 'cell', 'motorcycle'])

In [154]:
# Gender word lists from sueqian6's Github Repo "Reducing Gender Bias in Word-level Language Models"
# https://github.com/sueqian6/ACL2019-Reducing-Gender-Bias-in-Word-Level-Language-Models-Using-A-Gender-Equalizing-Loss-Function

gender_list_paths = glob.glob("./data/list/*_word_file.txt")
gender_list_paths.append('./data/list/neutral_occupations.txt')
gender_nouns_lookup = dict()
for path in gender_list_paths:
    with open(path, 'r') as f:
        if path == './data/list/neutral_occupations.txt':
            gender = 'neutral'
        else:
            p = re.compile(r'(?<=list/)(.*)(?=_word)')
            gender = p.findall(path)[0]
        nouns = f.read().split('\n')
        nouns = [n for n in nouns if n != '']
        gender_nouns_lookup[gender] = nouns

print(gender_nouns_lookup.keys())      

dict_keys(['female', 'male', 'neutral'])


In [155]:
# There are some inappropriate in the word lists, we are doing some hand crafting to avoid adding bias.
# Remove non-human words
words = ['cow', 'cows', 'hen', 'hens']
for word in words:
    gender_nouns_lookup['female'].remove(word)
words = ['bull', 'bulls', 'lion', 'lions', 'governor']
for word in words:
    gender_nouns_lookup['male'].remove(word)
    
# Add gender-neutral words
words = ['surfer', 'child', 'kid', 'kids', 'children', 'passenger', 'passengers',\
         'governor', 'someone', 'pedestrian', 'pedestrians']
for word in words:
    gender_nouns_lookup['neutral'].append(word)

In [216]:
def agreement_score(anno_gender):
    error = 0
    score_cal_dict ={
        'male':1, 'female':-1, 'neutral':0
    }
    gt = max(set(anno_gender), key = anno_gender.count)
    
    for ind, p in enumerate(anno_gender):
        for other_p in [x for i,x in enumerate(anno_gender) if i != ind]:
            error += np.abs(score_cal_dict[p] - score_cal_dict[other_p])
            
    score = (24 - error) / 24

    return score

In [297]:
annotations_path = './data/annotations/'
'''
captions_dict (dict)- key: image_id, value: list of captions

im_gender_summary (dict of dict)- key: image_id, value: dict()
keys in dict: pred_gt- predicted ground truth label of the gender noun
              per_gt- % of annotations (out of 5 total) that agreed with the GT
              agreement_score- agreement score calculated using distance between 5 predictions, with 1 being the best
                               male = 1, female = -1, neutral = 0
                               e.g.0 annotations indicate [f, f, f, f, f], agreement_score = 1.00
                               e.g.1 annotations indicate [m, m, f, f, f], agreement_score = 0.00
                               e.g.2 annotation indicate [n, n, f, f, f], agreement_score = 0.50                                                        
              anno_gender- list of gender sentiment, e.g. ['male', 'female', 'neutral', 'female', 'female']
              anno_nouns- list of nouns used to describe human
              clean_gender- binary variable indicating if all notations used the same gender/ gender-neutral noun 
              clean_noun- binary variable indicating if all notations used the identical noun

not_human_im_ids(list)- list of image ids of images with >1 captions that do not mention humans.
Since the COCO dataset does not label whether human (or other objects) is the major subject 
matter of the image. This list helps us isolate images with human figures as the focus.
'''
captions_dict = dict()
im_gender_summary = dict()
not_human_im_ids = list() 

for datatype in ['train', 'val']:
    print(f"Evaluating ground truth labels in {datatype} set')
        with open(f'{annotations_path}/captions_{datatype}2014.json') as f:
        captions_json = json.load(f)

        for i in range(len(captions_json['annotations'])):
            image_id = captions_json['annotations'][i]['image_id']
            caption = captions_json['annotations'][i]['caption']
            tokens = nltk.word_tokenize(caption)
            c_female = 0 # count of gender nouns and gender-neutral nouns
            c_male = 0
            c_neutral = 0
            noun = []

            # Evaluate annotator's noun used to describe humans
            for t in tokens:
                t = t.lower()
                if t in gender_nouns_lookup['female']:
                    c_female += 1
                    noun.append(t)
                elif t in gender_nouns_lookup['male']:
                    c_male += 1
                    noun.append(t)
                elif t in gender_nouns_lookup['neutral']:
                    c_neutral += 1
                    noun.append(t)

            # Only include image for training if more than one caption of the image mention human
            # Conflicting gender mentions are also dropped, e.g. "a boy and a girl are on a beach"
            if c_female + c_male + c_neutral == 1:
                # Assign gender sentiment to the caption
                if c_female > 0:
                    gender = 'female'
                elif c_male > 0:
                    gender = 'male'
                else:
                    gender = 'neutral'

                # Populate captions dict and image gender summary dict
                if image_id in captions_dict:
                    captions_dict[image_id] += [caption]
                    im_gender_summary[image_id]['anno_gender'].append(gender)
                    im_gender_summary[image_id]['anno_noun'].append(noun[0])
                else:
                    captions_dict[image_id] = [caption]
                    im_gender_summary[image_id] = dict()
                    im_gender_summary[image_id]['anno_gender'] = [gender]
                    im_gender_summary[image_id]['anno_noun'] = [noun[0]]

            if i % 100000 == 0:
                print()
                print(f"Caption {i} processed, out of {len(captions_json['annotations'])} captions")
                print(f"No. of qualified images processed: {len(im_gender_summary)}")



Caption 0 processed, out of 414113 captions
No. of qualified images processed: 0

Caption 100000 processed, out of 414113 captions
No. of qualified images processed: 6452

Caption 200000 processed, out of 414113 captions
No. of qualified images processed: 13359

Caption 300000 processed, out of 414113 captions
No. of qualified images processed: 24080

Caption 400000 processed, out of 414113 captions
No. of qualified images processed: 34712

Caption 0 processed, out of 202654 captions
No. of qualified images processed: 35500

Caption 100000 processed, out of 202654 captions
No. of qualified images processed: 42016

Caption 200000 processed, out of 202654 captions
No. of qualified images processed: 52292


In [298]:
for image_id in im_gender_summary:
    error = 0
    
    # Delete images where <3 annotators mentioned the human figure
    # Because it is impossible to estimate the ground truth using only 1 or 2 captions 
    if len(im_gender_summary[image_id]['anno_gender']) < 3:
        not_human_im_ids.append(image_id)
    
    else:
        pred = im_gender_summary[image_id]['anno_gender']

        # Evaluate groundtruth guesses and agreement scores
        gt = max(set(pred), key = pred.count)

        # Populate dictionary
        im_gender_summary[image_id]['pred_gt'] = gt
        im_gender_summary[image_id]['per_gt'] = sum([1 for p in pred if p == gt]) / len(pred)
        im_gender_summary[image_id]['agreement_score'] = agreement_score(pred)
        if len(set(pred)) == 1:
            im_gender_summary[image_id]['clean_gender'] = 1
        else:
            im_gender_summary[image_id]['clean_gender'] = 0
        if len(set(im_gender_summary[image_id]['anno_noun'])) == 1:
            im_gender_summary[image_id]['clean_noun'] = 1
        else:
            im_gender_summary[image_id]['clean_noun'] = 0
            
for image_id in not_human_im_ids:
    try:
        del captions_dict[image_id]
        del im_gender_summary[image_id]
    except:
        pass

In [299]:
for key in list(im_gender_summary.keys())[:5]:
    print(im_gender_summary[key])

{'anno_gender': ['female', 'female', 'female', 'female'], 'anno_noun': ['women', 'woman', 'woman', 'women'], 'pred_gt': 'female', 'per_gt': 1.0, 'agreement_score': 1.0, 'clean_gender': 1, 'clean_noun': 0}
{'anno_gender': ['female', 'female', 'female', 'female', 'female'], 'anno_noun': ['woman', 'woman', 'woman', 'woman', 'woman'], 'pred_gt': 'female', 'per_gt': 1.0, 'agreement_score': 1.0, 'clean_gender': 1, 'clean_noun': 1}
{'anno_gender': ['male', 'male', 'male', 'male'], 'anno_noun': ['man', 'man', 'man', 'man'], 'pred_gt': 'male', 'per_gt': 1.0, 'agreement_score': 1.0, 'clean_gender': 1, 'clean_noun': 1}
{'anno_gender': ['male', 'male', 'male'], 'anno_noun': ['man', 'man', 'man'], 'pred_gt': 'male', 'per_gt': 1.0, 'agreement_score': 1.0, 'clean_gender': 1, 'clean_noun': 1}
{'anno_gender': ['male', 'male', 'male', 'male', 'male'], 'anno_noun': ['man', 'man', 'man', 'man', 'man'], 'pred_gt': 'male', 'per_gt': 1.0, 'agreement_score': 1.0, 'clean_gender': 1, 'clean_noun': 1}


In [323]:
# Save list of image_ids of qualified images
with open('./data/list/qualified_image_ids.csv', "w") as outfile:
    for image_id in im_gender_summary.keys():
        outfile.write(str(image_id))
        outfile.write("\n")

In [321]:
def get_training_data(im_gender_summary, captions_dict, training_size, mode = 'random'):
    assert mode in ['random','balanced_mode','balanced_clean', 'balanced_gender_only', \
                    'balanced_clean_noun', 'clean_noun', 'activity_balanced', 'activity_balanced_clean']
    assert isinstance(training_size, int)
    '''
    8 different modes of generating data
    - random: randomized selection of qualified images
    - balanced_mode: balanced ratio between male, female and neutral
    - balanced_clean: balanced ratio between male, female and neutral,
                      only use images when all captions agree on using the same gender
    - balanced_gender_only: same as balanced_mode, but without neutral captions
    - balanced_clean_noun: balanced ratio between male, female and neutral, only use images when all captions
                           agree on using the same noun
    - clean_noun: only use images when all captions agree on the same noun
    - activity_balanced: from activity tagged image sets, choose same ratio of male, female, neutral image
    - activity_balanced_clean: similar to activity_balanced, but all captions must agree on the same gender
    
    Note that it is possible that output size may be smaller than training_size,
    especially for activity_balanced and activity_balanced_clean. As for certain activities, the sample size of
    clean data might be limited for some classes, e.g. women wearing tie.
    '''
    
    random.seed(123)
    training_captions_dict = dict()
    
    if mode == 'random':
        training_captions_dict = dict(random.sample(captions_dict.items(), training_size))
        
    elif mode == 'balanced_mode':
        i = 0
        male_count = 0
        female_count = 0
        neutral_count = 0
        for image_id in im_gender_summary.keys():
            if i < training_size:
                if im_gender_summary[image_id]['pred_gt'] == 'male' and (male_count < training_size / 3):
                    training_captions_dict[image_id] = captions_dict[image_id]
                    male_count += 1
                    i += 1
                elif im_gender_summary[image_id]['pred_gt'] == 'female' and (female_count < training_size / 3):
                    training_captions_dict[image_id] = captions_dict[image_id]
                    female_count += 1
                    i += 1
                elif im_gender_summary[image_id]['pred_gt'] == 'neutral'and (neutral_count < training_size / 3):
                    training_captions_dict[image_id] = captions_dict[image_id]
                    neutral_count += 1
                    i += 1
                    
                if i % 100 == 0:
                    print(f"captions of {i} images are added")
    
    elif mode == 'balanced_clean':
        i = 0
        male_count = 0
        female_count = 0
        neutral_count = 0
        for image_id in im_gender_summary.keys():
            if i < training_size:
                if im_gender_summary[image_id]['clean_gender'] == 1:
                    if im_gender_summary[image_id]['pred_gt'] == 'male' and (male_count < training_size / 3):
                        training_captions_dict[image_id] = captions_dict[image_id]
                        male_count += 1
                        i += 1
                    elif im_gender_summary[image_id]['pred_gt'] == 'female' and (female_count < training_size / 3):
                        training_captions_dict[image_id] = captions_dict[image_id]
                        female_count += 1
                        i += 1
                    elif im_gender_summary[image_id]['pred_gt'] == 'neutral'and (neutral_count < training_size / 3):
                        training_captions_dict[image_id] = captions_dict[image_id]
                        neutral_count += 1
                        i += 1
                    
                if i % 100 == 0:
                    print(f"captions of {i} images are added")
    
    elif mode == 'balanced_clean_noun':
        i = 0
        male_count = 0
        female_count = 0
        neutral_count = 0
        for image_id in im_gender_summary.keys():
            if i < training_size:
                if im_gender_summary[image_id]['clean_noun'] == 1:
                    if im_gender_summary[image_id]['pred_gt'] == 'male' and (male_count < training_size / 3):
                        training_captions_dict[image_id] = captions_dict[image_id]
                        male_count += 1
                        i += 1
                    elif im_gender_summary[image_id]['pred_gt'] == 'female' and (female_count < training_size / 3):
                        training_captions_dict[image_id] = captions_dict[image_id]
                        female_count += 1
                        i += 1
                    elif im_gender_summary[image_id]['pred_gt'] == 'neutral'and (neutral_count < training_size / 3):
                        training_captions_dict[image_id] = captions_dict[image_id]
                        neutral_count += 1
                        i += 1
                    
                if i % 100 == 0:
                    print(f"captions of {i} images are added") 
                    
    elif mode == 'clean_noun':
        i = 0
        for image_id in im_gender_summary.keys():
            if i < training_size:
                if im_gender_summary[image_id]['clean_noun'] == 1:
                    training_captions_dict[image_id] = captions_dict[image_id]
                    i += 1
                    
                if i % 100 == 0:
                    print(f"captions of {i} images are added")   
    
    elif mode == 'balanced_gender_only':
        i = 0
        male_count = 0
        female_count = 0
        for image_id in im_gender_summary.keys():
            if i < training_size:
                if im_gender_summary[image_id]['pred_gt'] == 'male' and (male_count < training_size / 2):
                    training_captions_dict[image_id] = captions_dict[image_id]
                    male_count += 1
                    i += 1
                elif im_gender_summary[image_id]['pred_gt'] == 'female' and (female_count < training_size / 2):
                    training_captions_dict[image_id] = captions_dict[image_id]
                    female_count += 1
                    i += 1
                    
                if i % 100 == 0:
                    print(f"captions of {i} images are added")
    
    elif mode == 'activity_balanced':
        activity_training_size = training_size / len(activity_image_ids.keys())
        i = 0
        for activity in activity_image_ids.keys():
            image_ids = activity_image_ids[activity]
            j = 0
            male_count = 0
            female_count = 0
            neutral_count = 0
            for image_id in image_ids:
                if j < activity_training_size:
                    if image_id in im_gender_summary:
                        if im_gender_summary[image_id]['pred_gt'] == 'male' and (male_count < activity_training_size / 3):
                            training_captions_dict[image_id] = captions_dict[image_id]
                            male_count += 1
                            i += 1
                            j += 1
                        elif im_gender_summary[image_id]['pred_gt'] == 'female' and (female_count < activity_training_size / 3):
                            training_captions_dict[image_id] = captions_dict[image_id]
                            female_count += 1
                            i += 1
                            j += 1
                        elif im_gender_summary[image_id]['pred_gt'] == 'neutral'and (neutral_count < activity_training_size / 3):
                            training_captions_dict[image_id] = captions_dict[image_id]
                            neutral_count += 1
                            i += 1
                            j += 1

                    if i > 0 and i % 100 == 0:
                        print(f"captions of {i} images are added")
    
    elif mode == 'activity_balanced_clean':
        activity_training_size = training_size / len(activity_image_ids.keys())
        i = 0
        for activity in activity_image_ids.keys():
            image_ids = activity_image_ids[activity]
            j = 0
            male_count = 0
            female_count = 0
            neutral_count = 0
            for image_id in image_ids:
                if j < activity_training_size:
                    if image_id in im_gender_summary and im_gender_summary[image_id]['clean_noun'] == 1:
                        if im_gender_summary[image_id]['pred_gt'] == 'male' and (male_count < activity_training_size / 3):
                            training_captions_dict[image_id] = captions_dict[image_id]
                            male_count += 1
                            i += 1
                            j += 1
                        elif im_gender_summary[image_id]['pred_gt'] == 'female' and (female_count < activity_training_size / 3):
                            training_captions_dict[image_id] = captions_dict[image_id]
                            female_count += 1
                            i += 1
                            j += 1
                        elif im_gender_summary[image_id]['pred_gt'] == 'neutral'and (neutral_count < activity_training_size / 3):
                            training_captions_dict[image_id] = captions_dict[image_id]
                            neutral_count += 1
                            i += 1
                            j += 1

                        if i > 0 and i % 100 == 0:
                            print(f"captions of {i} images are added")
    
    return training_captions_dict

In [322]:
training = get_training_data(im_gender_summary, captions_dict, training_size = 1000, mode = 'activity_balanced_clean')


captions of 100 images are added
captions of 200 images are added
captions of 300 images are added
captions of 400 images are added
captions of 500 images are added


492